<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2024/images/logo.png?raw=true" alt="2023年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

RNA-seqを用いた解析: Transcriptome解析
---

　今回から、ゲノム情報解析の中でも、RNA-seqを用いた解析を勉強・体験していきます。

RNA-seq解析シリーズ:
- RNA-seqとは？
- 遺伝子アノテーション
- 発現差異解析
- GO解析
- 遺伝子ネットワーク解析
- eQTL解析

等を扱います。

# 今回の勉強内容
　このテキストでは、RNA-seqを用いた基本的な解析について勉強します。

　内容は以下の通りです。

1. RNA-seqとは？
  1. mRNAをシーケンスする
  1. DNA-seqとRNA-seqの違い
1. 遺伝子のアノテーション
1. 発現解析
  1. 発現量の取得
  1. 正規化
  1. クラスタリング
  1. DEGの検出
  1. GO解析

---

　一度次のコードセルを実行して、実習に必要なライブラリのインストールや、プログラム、サンプルファイルのダウンロードしてください。

In [ ]:
#########################
##  実習の前に実行してください。 ##
#########################
%%bash
## ソフトウェアのインストール
pip install -q igv-notebook==0.3.1
pip list | grep -e "igv-notebook"
## サンプルファイルのダウンロード
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/scripts/igv_prep.py -O igv_prep.py
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/datasets/RNAseq_demo.tar.gz -O - | tar -zxf - 2>/dev/null

# 1. RNA-seqとは？

## mRNAをシーケンスする

これまで学んできた技術は、ある生物のDNA配列をシーケンスすることで、

生物のゲノム配列を読み取ったり、集団内での配列の違いをSNP等の形で検出することが出来る、という話でした。

また、この様なSNPの遺伝子型と形質値とを結び付けることで、QTLを見つけてきたり、形質値を予測したりすることが出来ました。

<br>ここからはRNAを対象としたシーケンス技術について学んでいきます。

遺伝子は、mRNAに転写された後、アミノ酸配列に翻訳され、アミノ酸配列が立体構造を取りタンパク質となることで何らかの機能が働きます。(セントラルドグマ)

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/central_dogma.png?raw=true" alt="central_dogma" height="280px" align="middle">

DNA配列を読み取るのがこれまで扱ってきたDNAシーケンスでしたが、RNAシーケンス(RNA-seq)では、転写されたmRNAを読み取ります。

RNA-seqによって、あるサンプルにおいて、どの遺伝子がどれだけ転写されているのか(遺伝子の発現量)、という情報を得ることが出来ます。

例えば、ある処理を行ったサンプルと行わなかったサンプルのRNA-seqを比較することで、処理によってどんな遺伝子が発現したのかが分かったりします。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/RNAseq_compare.png?raw=true" alt="RNAseq_compare" height="350px" align="middle">

## DNA-seqとRNA-seqの違い

では実際にサンプルデータを使って、DNAシーケンスとRNAシーケンスの違いを見てみましょう。

まずは最初の頃にやったDNAシーケンスのマッピング結果です。

In [ ]:
## DNAseqのアライメント結果を見る
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"RNAseq_demo/CultivarA.fa", "indexPath":"RNAseq_demo/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
A = BamTrack({ "name":"CultivarA", "path":"RNAseq_demo/CultivarA.bam", "indexPath":"RNAseq_demo/CultivarA.bam.bai", "viewAsPairs":True })
## IGV表示
b = igv_notebook.Browser(ref)
# gff file表示
b.load_track({"name": "Annotations", "type": "annotation", "format": "gff3", "displayMode": "EXPANDED", "path": "RNAseq_demo/CultivarA.gff3"})
b.load_track(A)

続いて、RNA-seqのマッピング結果です。

In [ ]:
## RNAseqのアライメント結果を見る
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"RNAseq_demo/CultivarA.fa", "indexPath":"RNAseq_demo/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
A = BamTrack({ "name":"CultivarA", "path":"RNAseq_demo/CultivarA_RNAseq.sort.bam", "indexPath":"RNAseq_demo/CultivarA_RNAseq.sort.bam.bai", "viewAsPairs":True })
## IGV表示
b = igv_notebook.Browser(ref)
## gff 表示
b.load_track({"name": "Annotations", "type": "annotation", "format": "gff3", "displayMode": "EXPANDED", "path": "RNAseq_demo/CultivarA.gff3"})
b.load_track(A)

Annotationsという行に表示されているのが、遺伝子の位置情報になります。exon領域が四角で表示されています。

この様にRNA-seqによって得られるリードをリファレンス配列にマッピングすると、exon領域のみに張り付いているのが分かるかと思います。

生物学の復習になりますが、mRNAに転写される際には、余分なもの(intron)を切り離してexonのみを再度つなぎ合わされるスプライシングが行われます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/splicing.png?raw=true" alt="splicing" height="250px" align="middle">

RNA-seqでは、転写されたmRNAをシーケンスするので、このexon領域をシーケンスしていることになります。

そのため、DNAシーケンスではゲノム全体からリードが得られるのに対して、RNA-seqで得られるリードは全てexon領域由来の物となります。

# 2. 遺伝子のアノテーション

それではRNA-seqによって得られたシーケンス情報をどのように活用していくのかを見ていきたいと思います。

## 遺伝子のアノテーション情報とは？

シーケンス技術の発展により、生き物のゲノム配列を解読することが可能となりました。

ある生き物のゲノム配列の情報だけでは無く、どの染色体のどの位置に遺伝子がコードされているのか、その遺伝子がどんな機能を持っているのか/持っていると考えられるのかといった情報をアノテーション情報と言います。

代表的な生物に関して、これまでの研究で解読されたゲノム配列やアノテーション情報がNCBI等を通じて公開されています。

NCBIの各生き物のデータベースにアクセスする[Taxonomy](https://www.ncbi.nlm.nih.gov/datasets/taxonomy/tree/)を使って例を見てみましょう。

好きな生き物の名前で検索してみて、もしその生き物のゲノム情報が登録されていれば「Reference genome」等の情報が表示されます。

その場合、「See in Genome Data Viewer」から各ゲノム領域の情報を確認することが出来ます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/NCBI_taxonomy.png?raw=true" alt="NCBI_taxonomy" height="500px" align="middle">

この様なゲノムデータはNCBIに限らず、イネだと[RAP-DB](https://rapdb.dna.affrc.go.jp/)等の様に、その植物専用のゲノム情報サイトが作られて公開されている場合もあります。

この様な遺伝子のアノテーション情報を決める際にRNA-seqが活用されます。

## RNA-seqによるアノテーション

DNA-seqによって全ゲノム配列がGATTATCCGCATCGGG...と分かっても、どのくらいの数の遺伝子がどこに位置しているのか、どの部分がexonなのか、intronなのか、といったより詳細なゲノム情報は全く分かりません。

そのため、遺伝子の情報を知るためには、別の情報を用いて遺伝子のアノテーションを行う必要があります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/unknown_reference.png?raw=true" alt="unknown_reference" height="300px" align="middle">

この遺伝子のアノテーションにRNA-seqを活用することが出来ます。

RNA-seqはexon領域のみをシーケンスすることになるので、マッピング結果を見て、RNA-seqによって得られたリードが張り付いた場所がexon領域だということが分かります。

先ほど見たRNA-seqの例から、遺伝子のexon, intronの位置情報を表示せずに見てみましょう。

In [ ]:
## RNAseqのマッピング結果を見る
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
ref = RefTrack({ "fastaPath":"RNAseq_demo/CultivarA.fa", "indexPath":"RNAseq_demo/CultivarA.fa.fai", "id":"CultivarA" })
A = BamTrack({ "name":"CultivarA", "path":"RNAseq_demo/CultivarA_RNAseq.sort.bam", "indexPath":"RNAseq_demo/CultivarA_RNAseq.sort.bam.bai", "viewAsPairs":True })
b = igv_notebook.Browser(ref)
b.load_track(A)

この様な結果から、ゲノム上でどこからどこまでがexon領域で、どこがintron領域か、ということを調べます。

また、exon領域にまたがるリードがいくつかあることが分かるかと思います。このことから、どのexonが結合して1つのmRNAとなっているか(どのexonが1つの遺伝子から来ているか)も調べることが出来ます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/gene_annotation_RNAseq.png?raw=true" alt="gene_annotation_RNAseq" height="350px" align="middle">

[StringTie](https://ccb.jhu.edu/software/stringtie/)等がこの様なアノテーションを行う有名なソフトになります。

### RNA-seqのマッピングについて

先ほどは触れませんでしたが、上述した様にRNA-seqのリードは複数のexon領域にまたがって張り付く場合があります。

そのため、DNA-seqと異なり、RNA-seqのリードをリファレンス配列にマッピングする際には、intronのギャップを考慮してマッピングする必要があります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/RNA_mapping.png?raw=true" alt="RNA_mapping" height="200px" align="middle">

そこで、[Hisat2](http://daehwankimlab.github.io/hisat2/)や[STAR](https://github.com/alexdobin/STAR)などの、RNA-seqに適した(ギャップを考慮してくれる)マッピングツールを使う必要があります。

## 他の遺伝子アノテーション方法

勿論、いつでも大量のRNA-seqデータが手に入るという訳ではないですし、その時RNAを抽出したサンプルで発現していない遺伝子のアノテーションは出来ません。

(そのため通常は色んな箇所や処理を行った様々なサンプルからRNAを抽出し、アノテーションに使用します。)

その様な場合には、配列情報をベースに遺伝子を予測する様な方法もあります。例えばAUG（メチオニン）が開始コドンであることは御存じだと思いますが、その様な情報を基に、どこからどこまでが遺伝子なのかを予測します。(ab initio gene predictionと呼んだりする。)

[AUGUSTUS](https://bioinf.uni-greifswald.de/augustus/)や[BRAKER](https://github.com/Gaius-Augustus/BRAKER)などが有名なプログラムになります。

また、近縁種で遺伝子がアノテーションされていた場合、それらの遺伝子情報を元に、同じような遺伝子が位置していないか探索するようなやり方もあります。(homology-based predictionと呼んだりする。)

例えばイネは"日本晴れ"という系統が最も遺伝子のアノテーション情報が整備されています。(例: [RAP-DB](https://rapdb.dna.affrc.go.jp/jbrowse/?data=data%2Firgsp1))

そこで、"こしひかり"という系統のゲノム配列を読み、遺伝子のアノテーションを行いたい時に、

まずはこの日本晴れの遺伝子情報を元に、ある程度似ている遺伝子や保存されている遺伝子について、こしひかりの遺伝子予測を行う、といった形です。



## gff/gtfファイルについて

上述した様々なアプローチを用いて遺伝子のアノテーションを行った結果は、GFF3フォーマットやGTFフォーマットと呼ばれる形式で保存されている場合が多いです。

(例)GFF3フォーマット

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/gff3_sample.png?raw=true" alt="gff3_sample" height="300px" align="middle">

代表的な植物種の遺伝子アノテーションの情報は[NCBI](https://www.ncbi.nlm.nih.gov/datasets/taxonomy/tree/)等のデータベースから入手することが出来ます。


# 3. 発現解析
次はRNA-seqを用いた最もポピュラーな解析である、遺伝子の発現解析について学びます。

先ほどの遺伝子のアノテーションについては、「ゲノム上のどこにRNA-seqのリードが張り付くか」を情報として扱っていましたが、次は「張り付くリードの量」を扱います。

RNA-seqによって、サンプル中のRNAをシーケンシングし、張り付くリードの量を調べることで、転写物の発現量(特定の遺伝子がmRNAにどれだけ転写されているのか)を定量することができます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/amount_transcripts.png?raw=true" alt="amount_transcripts" height="300px" align="middle">

そのため、通常のサンプルと何らかの処理をしたサンプルに対して RNA-seqを行い、発現量を比較することで、

処理の有無によって発現量の異なる転写物(遺伝子)を検出することができます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/deg_transcripts.png?raw=true" alt="deg_transcripts" height="300px" align="middle">

また、時系列サンプル(ある処理をして1h後、2h後、3h後...のそれぞれのサンプル等)に対して、RNA-seqを行うことで、

時間に応じて特定の遺伝子の発現がどの様に遷移するのかを明らかにできます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/time_transcripts.png?raw=true" alt="time_transcripts" height="300px" align="middle">

このように、様々なサンプルを用いてRNA-seqを行い、発現量解析を行うことで、

遺伝子の発現に対する**"量的な"**情報を得ることが出来ます。

## 発現量の取得

それでは、どの様に各遺伝子の発現量が定量化されるかを見ていきます。

基本的に生き物はその瞬間に必要な遺伝子をmRNAへ転写し、翻訳することでアミノ酸/タンパク質を合成している訳ですが、DNA配列が目で見えないのと同じように、今どんな遺伝子が発現している(/転写されているのか)も目では見えません。

そこで、RNA-seqを行うと、転写されたmRNAを読み取った大体数百万程のリードを得ることが出来ます。これらのリードを、先ほど紹介したソフト等を利用してマッピングすることで、どのリードがどの転写物(遺伝子)に由来するのかを決めることが出来ます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/where_RNA_reads.png?raw=true" alt="where_RNA_reads" height="500px" align="middle">

こうして、あるサンプルから得られたRNA-seqの全てのリードをマッピングし、各転写物毎のリードを数えることで、そのサンプルにおいて、どの遺伝子がどのくらい転写されていたかを数値化することが出来ます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/read_counts.png?raw=true" alt="read_counts" height="300px" align="middle">

[featureCounts](https://subread.sourceforge.net/)や[StringTie](https://ccb.jhu.edu/software/stringtie/)等のプログラムが転写物毎のリードのカウントを行ってくれます。

この様にして、あるサンプルの各遺伝子の発現量を定量化します。

実際に得られるデータを見てみましょう。

In [ ]:
#featureCountsの結果
import pandas as pd

counts = pd.read_csv("RNAseq_demo/feature_counts.txt", sep="\t", comment="#")
counts

## 正規化

とはいえ、マッピングされたリードの数をそのまま発現量として扱うことは通常しません。

なぜでしょうか？


### 正規化が必要な理由(サンプル内での比較の場合)

正規化が必要な場合は大きく分けて２つあります。

１つ目はサンプル内で複数の遺伝子の発現量を比較したい場合です。

例えば、ある生物の発現量を調べて、遺伝子AとBどちらが強く発現しているのかを調べたいとします。

このような場合、以下の様に単純にRNA-seqをして、マッピングされたリードの数を比べれば、どちらの遺伝子がよりよく転写されていたかを調べられそうです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/reads_AvsB.png?raw=true" alt="reads_AvsB" height="300px" align="middle">

しかし、このやり方では問題があります。

RNA-seqで得られるリードは、転写されたmRNAの断片になります。そのため、mRNAの長さが長ければ、シーケンシングされる確率が高く、得られるリードの数が多くなる傾向があります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/gene_length.png?raw=true" alt="gene_length" height="500px" align="middle">

よって、サンプル内の遺伝子の発現量を比較する場合には、遺伝子長に応じた補正が必要になります。

補正方法としては、FPKMやTPMと呼ばれる補正方法が代表的な手法になります。

### FPKM

FPKM(fragments per kilobase of exon per million reads mapped)では、ある遺伝子 i の発現量を

$FPKM_i = マッピングされたリード数 × 1000/転写産物の長さ × 1000000/全リード数$

という式で計算します。これは、まず遺伝子 i にマッピングされたリードの数を遺伝子の長さに応じて補正(遺伝子の長さが全て1000bpとなる様に揃える)し、その後、全リード数が1000000リードとなる様に補正していることになります。

### TPM

TPM(transcripts per million)では、ある遺伝子 i の発現量を、

1. 遺伝子 i の 1,000 bpあたりのリード数 $T_i$ を計算する。
$T_i = マッピングされたリード数 × 1000/転写産物の長さ$

2. 続いて、転写産物長による補正後の全リード数が100万となるように補正した値を発現量(TPM)として計算する。
$TPM_i = T_i × 1000000/\sum_{i}T_i$

これで、リード数を転写産物（遺伝子）の長さが 1 kbp あたりのリード数に換算してから、総リード数を 100 万に揃える補正が行われます。

FPKMとTPMどちらが良いのかという点ですが、基本的にはTPM を使用することが多いかと思います。

先ほどのカウントデータ(replicate1のみ)をTPMに変換してみます。

In [ ]:
# 得られたマッピング結果(リードカウント)と各遺伝子の長さ
counts.loc[:, ["Geneid", "Length", "replicate1"]]

まず、遺伝子が同じ長さ(=1000bp)と仮定したときのリード数$T$を求めます。

$T_i = マッピングされたリード数 × 1000/転写産物の長さ$

In [ ]:
# 遺伝子長で補正
counts["T"] = counts.loc[:, "replicate1"] * (1000/counts.Length)
counts.loc[:, ["Geneid", "Length", "replicate1", "T"]]

そして、総リード数が100万となるように補正したTPMを求めます。

In [ ]:
# 総リード数で補正
counts["TPM"] = counts["T"] * (1000000/sum(counts.T))
counts.loc[:, ["Geneid", "Length", "replicate1", "T", "TPM"]]

### 正規化が必要な場合(サンプル間での比較の場合)

２つ目の正規化が必要な場合としては、サンプル間で発現量を比較したい場合になります。

先ほどTPMを計算する際に、「総リード数が100万となる様に補正」していましたが、その理由がこの2つ目です。

例えば、通常時のサンプルと、ある処理を行ったサンプルの遺伝子の発現量を比較したいとします。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/reads_samples.png?raw=true" alt="reads_samples" height="450px" align="middle">

この場合、それぞれのサンプルでRNA-seqを行い、マッピング結果を元に比較することになりますが、この時もそのままのリード数を発現量としてはいけません。

サンプル間の総リード数を補正する必要があるからですね。

下に示すように、RNA-seqによって得られたリードの数が異なると、マッピングされるリードの数も異なってきます。

そのため、同じくらいのリード数が得られた場合に、どのくらいリード数があるのか、といった形に補正する必要があります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/reads_samples2.png?raw=true" alt="reads_samples2" height="450px" align="middle">

最も簡単な補正方法は、CPM(count per million)と呼ばれる、全リード数が100万になる様に揃える補正方法になります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/CPM.png?raw=true" alt="CPM" height="150px" align="middle">

ただし、実際にサンプル間の比較をする際には、TMM正規化やRLE正規化と呼ばれるCPMを更に補正した方法が使われることが多いです。

これらの補正方法は、ハウスキーピング遺伝子と呼ばれる、常に一定の発現量を示すとされている遺伝子群に着目した手法です。ハウスキーピング遺伝子の発現量がサンプル間で揃うように、補正を行う形になります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/TMM.png?raw=true" alt="TMM" height="160px" align="middle">

これらの発現量の補正にはRの`edgeR`や`DESeq`、`DEseq2`といったパッケージを活用することが多いです。

この様な形で、元々のリード数から補正されたリード数(発現量)を計算して、遺伝子間の比較を行います。

### サンプル間の比較

次にサンプル間で発現量の比較をする場合を見てみます。主に発現解析を行う目的はこちらになります。

以下に示したデータは、イネに水もしくは病原菌を接種して２４時間後の遺伝子の発現量をTMM値として表示しています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/DEG_data.png?raw=true" alt="DEG_data" height="320px" align="middle">

In [ ]:
# サンプル間での遺伝子発現比較(TMM)
TMM = pd.read_csv("RNAseq_demo/TMM.txt", sep="\t")
TMM

### DEGの検出

サンプル間で遺伝子の発現量を比較する場合、発現量が異なる遺伝子DEG(Differentially expressed gene)を検出することが大きな目的の一つになります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/DEG.png?raw=true" alt="DEG" height="400px" align="middle">

通常、DEGを検出するためには、各サンプルのRNA-seqデータの反復を取った上で、サンプル間で遺伝子毎に発現量に差があるかの検定を行う必要があります。

詳細は省きますが、尤度比検定(edgeRやDESeq2で実装)やWald検定(DESeq2で実装)などが主に用いられます。

また、当然ですが遺伝子の数だけ検定を行う形になるので、多重検定の補正を行う必要もあります(edgeRやDESeqで実装)。

In [ ]:
# 検定結果(DESeq2の場合)
DESeq2 = pd.read_csv("RNAseq_demo/DESeq2.txt", sep="\t", index_col=0)
DESeq2

これは全ての遺伝子に対して検定を行った結果なので、

この中から有意だったもの(adjusted p値 = q値が0.05未満、かつ	log2FoldChangeが1以上)を取り出すと…

In [ ]:
DESeq2[(DESeq2.log2FoldChange.abs() >= 1) & (DESeq2.padj < 0.05)].sort_values(by="padj")

今回の場合、Q値 < 0.05かつlog2FoldChangeの絶対値が1以上(２倍以上発現量が変わっている)を基準として1583個の遺伝子がDEGとして検出された形になります。

例を見てみると

In [ ]:
TMM[TMM["Geneid"] == "Gene.12504"]

### クラスタリング

各サンプルの遺伝子全体の発現量の関係性を確かめる方法の一つにクラスタリングがあります。

RNA-seqを行う際、処理を行ったサンプルや通常のサンプル、それらの反復など、複数のサンプルを用いる場合が多いですが、それらのデータを使って発現解析を行う前にクラスタリングを行うことで、きちんと処理の有無でデータがきれいに分かれているか等、各サンプルの特徴を確認することが出来ます。

実験が上手くいっているかの確認をしたり、発現パターンがどのくらいあるのかを確認するためにクラスタリングを行うことも多いです。

先ほどのイネに水もしくは病原菌を接種したRNA-seqデータを使ってクラスタリングしてみます。

In [ ]:
# DEGのクラスタリング結果
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

DESeq2 = pd.read_csv("RNAseq_demo/DESeq2.txt", sep="\t", index_col=0)
DESeq2 = DESeq2[(DESeq2.log2FoldChange.abs() >= 1) & (DESeq2.padj < 0.05)].sort_values(by="padj")

TMM_values = pd.read_csv("RNAseq_demo/TMM.txt", sep="\t", index_col=0)
sns.clustermap(TMM_values.loc[DESeq2.index, :], vmin=0, vmax=100, cmap="mako")

発現量データを元にしたクラスタリングの手法としては、階層的クラスタリング、k-means、主成分分析(PCA)など様々な手法があります。

### GO解析

DEGを検出したり、クラスタリング解析を行う時に、一緒に行われることが多い解析がGO解析(Gene ontology解析)です。

Gene ontology(GO)とは、遺伝子の生物的プロセス、細胞の要素や分子機能に関して、遺伝子に付けられるアノテーション情報です。遺伝子Xは防御応答に関与している、とか、遺伝子の転写制御に関わっている等の情報がGOとして予測されています。

例:イネの`Os10g0549500`という遺伝子には
* `GO:0004084(branched-chain-amino-acid transaminase activity)`
* `GO:0052656(L-isoleucine transaminase activity)`
* `GO:0044281(small molecule metabolic process)`
* ...

とGO の情報がつけられており、着目する遺伝子のGOを調べることで、その遺伝子の機能や働き等を推定できます。

#### GO enrichment解析

GO解析の中でも、GO enrichment解析は、ある遺伝子セットのGOを解析することで、特にその遺伝子セットによくみられる機能等を調べる解析になります。

先ほどの例で言うと、DEG解析で病原菌を接種した際に発現量が上昇する遺伝子群を調べ、その遺伝子群に対しGO解析を行うことで発現量が上がった遺伝子群がどの様な機能を持つ遺伝子が多かったのか調べる、の様な形です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GO_enrichment2.png?raw=true" alt="GO_enrichment2" height="400px" align="middle">

ただ、あくまでもそういう傾向がある、という情報しか分からないので、よく見かける解析ではありますが(個人的に)あんまり好きじゃない解析です。

GO解析はRのパッケージ`GO.db`, `clusterProfiler`, `topGO`等で行うことが出来ます。

また、ヒトやマウス、シロイヌナズナ、イネなどの代表的な生物であれば、web上でGO解析を行うことが出来るサイトもあります。

[g:Profiler](https://biit.cs.ut.ee/gprofiler/gost)などが有名でしょうか。



# まとめ

　今回、RNA-seqを活用した基本的な解析の流れや注意点をみてきました。
RNA-seqを活用することで、遺伝子のアノテーションを行ったり、ある生物の遺伝子の発現を調べることが出来ます。

特に、発現データを処理サンプルのデータと比較することによって、処理によって発現変動を起こす遺伝子群の特定をすることが出来ます。

どの様な遺伝子がどのような時に発現しているのかを知るには重要なアプローチの１つです。

次回からはRNA-seqを用いてネットワーク解析と呼ばれるものをやってみます。

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>